In [93]:
pip install opencv-python pyOpenSSL --upgrade --upgrade youtube_dl install pafy install pytube pytesseract setuptools wheel spacy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.6.0
    Uninstalling setuptools-67.6.0:
      Successfully uninstalled setuptools-67.6.0
  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 23.1.0
    Uninstalling pyOpenSSL-23.1.0:
      Successfully uninstalled pyOpenSSL-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python3 -m spacy download es_core_news_lg

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [96]:
import cv2
import os
import numpy as np
import pytesseract
from pytesseract import Output
from pytube import YouTube

import spacy
import re

In [97]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [99]:
class Video:
    def __init__(self, path, url):
        self.path = path
        self.videoUrl = url
        self.yt = self.Download()
        #self.localUrl = ''
        #self.yt = ''
        
    def Download(self):
        #por ahora para archivos de internet
        file = self.Network()
        print(file)
        return file

    def Network(self):
        yt = YouTube(self.videoUrl)
        yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
        if not os.path.exists(self.path):
            os.makedirs(self.path)
        localUrl = yt.download(self.path)
        self.localUrl = localUrl
        return yt

#'https://www.youtube.com/watch?v=REAFtXGnpKU'


In [144]:
nlp = spacy.load("es_core_news_lg")

# Verificamos si la palabra existe en el vocabulario del modelo
def spanish_word(word):
    if nlp.vocab.has_vector(word):
        return True
    else:
        return False

def word_validate(sentence):
    doc_es = nlp(sentence)
    words =  []

    # Expresión regular que busca cualquier caracter que no sea una letra, número o espacio en blanco
    patron = r'[^a-zA-Z1-9ñÑáéíóúÁÉÍÓÚüÜ\d\s/\d.]'
    validate = ['NUM', 'PUNCT']

    for token in doc_es:

        # Buscamos el patrón en el texto
        res = re.search(patron, token.text)

        # Si se encuentra algún símbolo devuelve True
        is_symbol = res is not None

        if not is_symbol:
            if token.tag_ not in validate:

                is_spanish = spanish_word(token.text)

                if is_spanish :
                    words.append(token.text)
            else:
                words.append(token.text)

    return words


In [149]:

video = Video('C:/Users/jasura/Documents/maestria/seminario',
 'https://www.youtube.com/watch?v=XE6epSyQrkw' )


<Stream: itag="22" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">


In [102]:
pip install -U matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [103]:
video.localUrl

'/Users/jasura/Documents/maestria/seminario/C:/Users/jasura/Documents/maestria/seminario/Masa para tacos wraps  Rapiditas Caseras - CUKit!.mp4'

In [104]:
from PIL import Image, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt

In [151]:
cam = cv2.VideoCapture(video.localUrl)

In [134]:
def img_process(img):

    #convertimos a RGB
    color_coverted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
    # Displaying the converted image
    image = Image.fromarray(color_coverted)

    # agregado de sharpness y contrast a la  imagen
    enhancer1 = ImageEnhance.Sharpness(image)
    enhancer2 = ImageEnhance.Contrast(image)
    img_edit = enhancer1.enhance(20.0)
    img_edit = enhancer2.enhance(1.5)

    cv_img = cv2.cvtColor(np.array(img_edit), cv2.COLOR_RGB2BGR)

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    
    # Aplicar una operación morfológica para eliminar el ruido
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    
    # Aplicar una operación de threshold para aumentar el contraste
    thresh = cv2.threshold(opening, 200, 255, cv2.THRESH_BINARY_INV)[1]
    #plt.imshow(thresh);
    # Encontrar contornos en la imagen thresholdada
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Inicializar una lista para almacenar los rectángulos que contienen el texto
    rects = []
    
    # Recorrer cada contorno
    for cnt in contours:
        # Obtener el rectángulo que encierra el contorno
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Agregar el rectángulo a la lista si cumple con ciertos criterios
        if w > 50 and h > 50:
            rects.append((x, y, w, h))


    # Recorrer cada rectángulo
    for rect in rects:
        x, y, w, h = rect
        roi = thresh[y:y+h, x:x+w]
        
    #convertimos a RGB
    color_coverted = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
  
    # Displaying the converted image
    pil_image = Image.fromarray(color_coverted)

    return pil_image

In [154]:
def extract_text_img(img):

    # Inicializar una lista para almacenar el texto extraído
    text = []
    black_list = ['', ' ', ' \n']

    image = img_process(img)

    # Utilizar OCR (Reconocimiento Óptico de Caracteres) para extraer el texto de la región de interés
    ocr = pytesseract.image_to_string(image)

    #print(ocr, len(ocr))

    ocr_validate = word_validate(ocr)

    if len(ocr_validate) == 0:
        return None

    #print(text)
    return ' '.join(ocr_validate)


In [155]:

complete_text = []
prev_text = ""

# Establecer un nivel mínimo de similitud exigido:
nivel_minimo = 0.7



while(cam.isOpened()):
    # Leer el siguiente fotograma
    ret, frame = cam.read()

    # color_coverted = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  
    
    # # Displaying the converted image
    # pil_image = Image.fromarray(color_coverted)

    # Si no hay más fotogramas, salir del bucle
    if not ret:
        break

    # Validamos que la imagen no sea 100% negra
    is_black = not np.any(frame)

    #si la img es negra ignoramos
    if is_black:
        continue

    # Utilizar funcion extract_text extraer el texto de la imagen
    text = extract_text_img(frame)

    if text :

        print(prev_text," - ", text)
        # Obtener el objeto Doc para cada texto:
        pt = nlp(prev_text)
        nt = nlp(text)

        # Calcular la similitud entre los textos:
        similitud = pt.similarity(nt)


        if similitud >= nivel_minimo:
            #print('Los textos son similares.')
            continue
        else:
            #print('Los textos no son suficientemente similares.')
            prev_text = text
            complete_text.append(text)
    
    # Imprimir el texto extraído

print(' '.join(complete_text))
# Liberar la cámara
cam.release()

  -  HARINA 0000 250GR . A.


/var/folders/fk/k827gpdx4qg96rvjc3n5s7b80000gn/T/ipykernel_2952/506074279.py:41: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similitud = pt.similarity(nt)


HARINA 0000 250GR . A.  -  A. HARINA 0000 250 GR .
HARINA 0000 250GR . A.  -  HARINA 0000 250 GR .
HARINA 0000 250 GR .  -  HARINA 0000 250GR .
HARINA 0000 250GR .  -  HARINA 0000 250 GR .
HARINA 0000 250 GR .  -  pa ta
pa ta  -  Ay HARINA 0000 250 GR .
Ay HARINA 0000 250 GR .  -  HARINA 0000 250 GR .
Ay HARINA 0000 250 GR .  -  A HARINA 0000 250GR .
A HARINA 0000 250GR .  -  HARINA 0000 250 GR . La
HARINA 0000 250 GR . La  -  HARINA 0000 250 GR . e
HARINA 0000 250 GR . e  -  HARINA 0000 250 GR . bd
HARINA 0000 250 GR . e  -  HARINA 0000 250 GR . e


KeyboardInterrupt: 